# Gruppo 8A
Ennio Malvati - 484998 <br>
Gianluca Di Lorenzo - 583630 <br>
Raffaele Scarano - 576304

# Test `spatial_arrangment_1`

In questa prima fase vengono caricate tutte le librerie utili per l'implementazione e per la fase di test

In [61]:
using Pkg
Pkg.activate("./../../TopologicalGiftWrapping8A")
using Profile
using ProfileView
using BenchmarkTools
using SparseArrays
using LinearAlgebra
using Distributed
using LinearAlgebraicRepresentation
using NearestNeighbors
Lar = LinearAlgebraicRepresentation
using ViewerGL
GL = ViewerGL

  Activating project at `~/Università/Calcolo_parallelo/TopologicalGiftWrapping8A`


ViewerGL

Successivamente viene preparato l'input dove V e' di tipo Matrix-Real con tre righe e colonne pari alla cardinalita' di C0, inoltre, EV ed FE sono matrici sparse di co-bordi e forniscono le incidenze tra i bordi, i vertici e le facce

In [62]:
function get_input()
    V = [0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; 0.5941251 0.8769965 0.3624924; 0.6542998 0.8170598 1.0344647; 1.3260341 0.2707609 0.2428771; 1.3862088 0.2108241 0.9148494; 1.2660973 0.9422582 0.3081388; 1.326272 0.8823214 0.980111; -0.3874063 0.4902226 0.4536339; 0.3249123 0.707347 0.5231232; -0.1702819 -0.0864242 0.0297177; 0.5420367 0.1307001 0.099207; -0.317917 0.0663064 1.0658723; 0.3944016 0.2834308 1.1353616; -0.1007926 -0.5103404 0.6419561; 0.611526 -0.2932161 0.7114454; 0.7899026 0.0605793 0.6679889; 0.46601 0.0749997 0.6686316; 0.804323 0.3844725 0.6679746; 0.4804304 0.3988929 0.6686173; 0.7905452 0.060565 0.9922023; 0.4666527 0.0749854 0.992845; 0.8049656 0.3844582 0.992188; 0.4810731 0.3988786 0.9928307; -0.2261907 -0.0720455 0.4715635; -0.0499888 0.0863489 0.7965885; -0.0677963 0.219164 0.24378; 0.1084056 0.3775584 0.5688049; 0.0988343 -0.2998291 0.4063673; 0.2750362 -0.1414347 0.7313923; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087];

    EV = SparseArrays.sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2, 6, 12, 3, 7, 9, 3, 8, 10, 4, 7, 11, 4, 8, 12, 13, 17, 21, 13, 18, 22, 14, 17, 23, 14, 18, 24, 15, 19, 21, 15, 20, 22, 16, 19, 23, 16, 20, 24, 25, 29, 33, 25, 30, 34, 26, 29, 35, 26, 30, 36, 27, 31, 33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10, 11, 11, 11, 12, 12, 12, 13, 13, 13, 14, 14, 14, 15, 15, 15, 16, 16, 16, 17, 17, 17, 18, 18, 18, 19, 19, 19, 20, 20, 20, 21, 21, 21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, -1, -1, -1, 1, -1, -1, -1, 1, -1, 1, 1, -1, -1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1]);

    FE = SparseArrays.sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5, 1, 6, 2, 5, 2, 6, 3, 5, 3, 6, 4, 5, 4, 6, 7, 9, 7, 10, 8, 9, 8, 10, 7, 11, 7, 12, 8, 11, 8, 12, 9, 11, 9, 12, 10, 11, 10, 12, 13, 15, 13, 16, 14, 15, 14, 16, 13, 17, 13, 18, 14, 17, 14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10, 10, 11, 11, 12, 12, 13, 13, 14, 14, 15, 15, 16, 16, 17, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24, 24, 25, 25, 26, 26, 27, 27, 28, 28, 29, 29, 30, 30, 31, 31, 32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, -1, 1, 1, -1, -1, -1, -1, 1, 1, 1, -1, -1, 1, -1, -1, -1, 1, -1, -1, 1, 1, 1]);
    return V, EV, FE
end

get_input (generic function with 1 method)

Testiamo il tempo d'esecuzione della funzione `spatial_arrangment` utilizzando la macro `@btime` la quale ci permette di valutare svariate volte le prestazioni dal punto di vista temporale delle funzioni al fine di ridurre il rumore.

In [63]:
V, EV, FE = get_input()
@btime Lar.Arrangement.spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

  12.658 ms (225321 allocations: 11.94 MiB)


([0.6540618000000001 0.2054992 0.2972308; 0.7142365000000002 0.1455625 0.969203; … ; 0.8049656 0.3844582 0.992188; 0.4810731000000001 0.3988786 0.9928307], sparse([1, 4, 14, 2, 5, 15, 1, 2, 7, 17  …  57, 61, 65, 53, 56, 63, 55, 58, 64, 66], [1, 1, 1, 2, 2, 2, 3, 3, 3, 3  …  36, 36, 36, 37, 37, 37, 38, 38, 38, 38], Int8[-1, -1, -1, -1, -1, -1, 1, 1, -1, -1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 67, 38), sparse([1, 4, 2, 5, 1, 6, 1, 7, 2, 8  …  25, 29, 27, 28, 26, 31, 32, 31, 30, 31], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  62, 62, 63, 63, 64, 64, 65, 66, 67, 67], Int8[1, 1, -1, -1, -1, 1, -1, 1, 1, 1  …  1, -1, -1, 1, 1, 1, 1, -1, -1, 1], 32, 67))

In [64]:
V, EV, FE = get_input()
@benchmark Lar.Arrangement.spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

BenchmarkTools.Trial: 355 samples with 1 evaluation.
 Range (min … max):  12.617 ms … 23.490 ms  ┊ GC (min … max): 0.00% … 31.90%
 Time  (median):     12.814 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.097 ms ±  2.710 ms  ┊ GC (mean ± σ):  7.22% ± 12.06%

  █▇▂                                         ▃▂               
  █████▆▇▆▁█▅▅▇▆▅▄▅▄▁▁▁▁▄▁▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▄▁██▅▄▁▁▁▁▁▁▁▁▁▅▄ ▆
  12.6 ms      Histogram: log(frequency) by time      22.4 ms <

 Memory estimate: 11.94 MiB, allocs estimate: 225321.

Utilizzando la macro `@code_warntype` ci permette di generare una rappresentazione del codice che può essere utile per trovare espressioni che determinano l'incertezza del tipo, utile per valutare la tipicizzazione di una funzione.
La caratteristica più importante di @code_warntype è che i tipi non concreti sono visualizzati in rosso. Per facilitare la ricerca delle instabilità di tipo che potrebbero essere importanti, le unioni che contengono `missing` o `nothing` sono evidenziate in giallo, invece che in rosso.

In [65]:
V, EV, FE = get_input()
@code_warntype Lar.Arrangement.spatial_arrangement_1(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),true)

MethodInstance for LinearAlgebraicRepresentation.Arrangement.spatial_arrangement_1(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Bool)
  from spatial_arrangement_1(V::Matrix, copEV::SparseMatrixCSC{Int8, Int64}, copFE::SparseMatrixCSC{Int8, Int64}, multiproc::Bool) in LinearAlgebraicRepresentation.Arrangement at /Users/raffaelescarano/.julia/packages/LinearAlgebraicRepresentation/qwzGL/src/arrangement/spatial_arrangement.jl:143
Arguments
  #self#::Core.Const(LinearAlgebraicRepresentation.Arrangement.spatial_arrangement_1)
  V::Matrix{Float64}
  copEV::SparseMatrixCSC{Int8, Int64}
  copFE::SparseMatrixCSC{Int8, Int64}
  multiproc::Bool
Locals
  @_6::Int64
  @_7::Union{Nothing, Tuple{Int64, Int64}}
  @_8::Union{Nothing, Tuple{Int64, Int64}}
  @_9::Union{Nothing, Tuple{Int64, Int64}}
  #81::LinearAlgebraicRepresentation.Arrangement.var"#81#85"
  #80::LinearAlgebraicRepresentation.Arrangement.var"#80#84"
  rFE::Any
  rEV::Any
  rV::Any
  out_chan::Any

│    %106 = Base.not_int(%105)::Bool
└───        goto #16 if not %106
14 ┄ %108 = @_7::Tuple{Int64, Int64}
│           (sigma@_36 = Core.getfield(%108, 1))
│    %110 = Core.getfield(%108, 2)::Int64
│    %111 = Base.getproperty(LinearAlgebraicRepresentation.Arrangement.Lar, :Arrangement)::Any
│    %112 = Base.getproperty(%111, :frag_face)::Any
│    %113 = sp_idx::Any
│    %114 = (%112)(V, copEV, copFE, %113, sigma@_36)::Any
│    %115 = Base.indexed_iterate(%114, 1)::Any
│           (nV = Core.getfield(%115, 1))
│           (@_29 = Core.getfield(%115, 2))
│    %118 = Base.indexed_iterate(%114, 2, @_29)::Any
│           (nEV = Core.getfield(%118, 1))
│           (@_29 = Core.getfield(%118, 2))
│    %121 = Base.indexed_iterate(%114, 3, @_29)::Any
│           (nFE = Core.getfield(%121, 1))
│    %123 = Base.getproperty(LinearAlgebraicRepresentation.Arrangement.Lar, :Points)::Any
│           (nV = LinearAlgebraicRepresentation.Arrangement.convert(%123, nV))
│    %125 = Base.getproperty(Linear

# Ottimizazzione `frag_face`

La funzione `spatial_arrangment_1` utilizza al suo interno, in un ciclo `for`, la funzione `frag_face` quindi una sua eventuale ottimizzazione è auspicabile. Tale funzione effettua la trasformazione in 2D delle facce fornite in input tramite il parametro sigma ed infine ogni faccia di questo parametro sigma si interseca con le facce `insp_index`. 

Con la funzione `@btime` si valutano le prestazioni dal punto di vista temporale della funzione.

In [66]:
V, EV, FE = get_input()
@btime Lar.Arrangement.frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

  225.375 μs (4803 allocations: 265.63 KiB)


([1.3260341 0.2707609000000001 0.24287710000000004; 1.3862088 0.21082410000000004 0.9148494000000003; 1.2660973 0.9422582000000003 0.30813880000000005; 1.3262719920385087 0.8823213992267827 0.9801110006439793], sparse([1, 3, 1, 4, 2, 3, 2, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, -1, 1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, -1, -1, 1], 1, 4))

Utilizzando la macro `@code_warntype` possiamo valutare la tipicizzazione della funzione.

In [67]:
@code_warntype Lar.Arrangement.frag_face(Lar.Points(V),EV,FE,[2,3,4,5],2)

Excessive output truncated after 524288 bytes.

MethodInstance for LinearAlgebraicRepresentation.Arrangement.frag_face(::Matrix{Float64}, ::SparseMatrixCSC{Int8, Int64}, ::SparseMatrixCSC{Int8, Int64}, ::Vector{Int64}, ::Int64)
  from frag_face(V, EV, FE, sp_idx, sigma) in LinearAlgebraicRepresentation.Arrangement at /Users/raffaelescarano/.julia/packages/LinearAlgebraicRepresentation/qwzGL/src/arrangement/spatial_arrangement.jl:27
Arguments
  #self#::Core.Const(LinearAlgebraicRepresentation.Arrangement.frag_face)
  V::Matrix{Float64}
  EV::SparseMatrixCSC{Int8, Int64}
  FE::SparseMatrixCSC{Int8, Int64}
  sp_idx::Vector{Int64}
  sigma::Int64
Locals
  @_7::Int64
  @_8::Union{Nothing, Tuple{Int64, Nothing}}
  nvsize::Any
  nFE::Any
  nEV::Any
  nV::Any
  tV::Any
  M::Any
  sEV::Any
  sV::Any
  sigmavs::Vector{Int64}
  vs_num::Int64
  @_19::Any
  @_20::Any
  i::Int64
  tmpEV::Any
  tmpV::Any
Body::Tuple{Any, Any, Any}
1 ─       Core.NewvarNode(:(@_7))
│         Core.NewvarNode(:(nvsize))
│         Core.NewvarNode(:(nFE))
│         Core

# Ottimizzazione `frag_face`

Riportiamo la versione parallelizzata di `frag_face`

In [68]:
function frag_face_parallelized(V::Matrix{Float64}, EV::SparseMatrixCSC{Int8, Int64}, FE::SparseMatrixCSC{Int8, Int64},
    sp_idx::Vector{Vector{Int64}}, sigma::Int64)

    vs_num::Int64 = size(V, 1)
    
	# 2D transformation of sigma face
    sigmavs::Vector{Int64} = (abs.(FE[sigma:sigma,:])*abs.(EV))[1,:].nzind
    
    sV::Matrix{Float64} = V[sigmavs, :]
    sEV::SparseMatrixCSC{Int8, Int64} = EV[FE[sigma, :].nzind, sigmavs]
    M::Matrix{Float64} = Lar.Arrangement.submanifold_mapping(sV)
    tV::Matrix{Float64} = ([V ones(vs_num)]*M)[:, 1:3]
    sV = tV[sigmavs, :]
    
    # sigma face intersection with faces in sp_idx[sigma]
    @async for i in sp_idx[sigma]
        tmpV::Matrix{Any}, tmpEV::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.face_int(tV, EV, FE[i, :])
        sV, sEV = Lar.skel_merge(sV, sEV, tmpV, tmpEV)
    end

    # computation of 2D arrangement of sigma face
    sV = sV[:, 1:2]
    nV::Matrix{Float64}, nEV::SparseMatrixCSC{Int8, Int64}, nFE::SparseMatrixCSC{Int8, Int64} = Lar.Arrangement.planar_arrangement(sV, sEV, sparsevec(ones(Int8, length(sigmavs))))
    
    nvsize::Int64 = size(nV, 1)
    nV = [nV zeros(nvsize) ones(nvsize)]*inv(M)[:, 1:3]
    
    return nV, nEV, nFE
end

frag_face_parallelized (generic function with 2 methods)

Utilizziamo la macro `@btime` per valutare le prestazioni dal punto di vista temporale e capire se le prestazioni della funzione `frag_face` sono migliorate.

In [69]:
V, EV, FE = get_input()
@btime frag_face_parallelized(Lar.Points(V),EV,FE,[2,3,4,5],2)

([1.3260341 0.2707609000000001 0.24287710000000004; 1.3862088 0.21082410000000004 0.9148494000000003; 1.2660973 0.9422582000000003 0.30813880000000005; 1.3262719920385087 0.8823213992267827 0.9801110006439793], sparse([1, 3, 1, 4, 2, 3, 2, 4], [1, 1, 2, 2, 3, 3, 4, 4], Int8[-1, -1, 1, -1, -1, 1, 1, 1], 4, 4), sparse([1, 1, 1, 1], [1, 2, 3, 4], Int8[1, -1, -1, 1], 1, 4))

Utilizziamo la macro `@profile` la quale esegue l’espressione a cui è assegnata collezionando periodicamente dei campioni in modo tale da mostrare la gerarchia delle funzioni ed il tempo di esecuzione di ogni riga.

In [70]:
Profile.clear()
Profile.init(delay=.1e-7)
@profile for _ in 1:200
    frag_face_parallelized(Lar.Points(V),EV,FE,[2,3,4,5],2)
end

In [71]:
ProfileView.view()
Profile.print()

# Valutazione `spatial_arrangment_1`

La valutazione viene fatta aggiungendo al codice del precedente metodo `spatial_arrangment_1` la funzione `frag_face_parallelized`

In [72]:
function spatial_arrangement_1_with_frag(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = frag_face_parallelized(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = Lar.Arrangement.merge_vertices(rV, rEV, rFE)
  return rV, rEV, rFE
end

spatial_arrangement_1_with_frag (generic function with 2 methods)

Con la funzione `@btime` valutiamo le prestazioni dal punto di vista temporale della funzione.

In [73]:
V, EV, FE = get_input()
@btime spatial_arrangement_1_with_frag(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

([0.6540618000000001 0.2054992 0.2972308; 0.7142365000000002 0.1455625 0.969203; … ; 0.8049656 0.3844582 0.992188; 0.4810731000000001 0.3988786 0.9928307], sparse([1, 3, 9, 1, 4, 10, 2, 3, 11, 2  …  33, 29, 32, 34, 30, 31, 35, 30, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 24), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, -1, 1, 1, 1, 1, -1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

In [74]:
V, EV, FE = get_input()
@benchmark spatial_arrangement_1_with_frag(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false)

BenchmarkTools.Trial: 1146 samples with 1 evaluation.
 Range (min … max):  3.432 ms … 21.629 ms  ┊ GC (min … max):  0.00% … 75.56%
 Time  (median):     3.675 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.361 ms ±  2.971 ms  ┊ GC (mean ± σ):  13.14% ± 14.98%

  █▆▄▂▁                                                       
  █████▆▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▃▄▅▃▆▅▄▃▁▄▆▆ ▇
  3.43 ms      Histogram: log(frequency) by time       20 ms <

 Memory estimate: 4.58 MiB, allocs estimate: 82815.

# Analisi e ottimizzazione `merge_vertices`

La funzione `merge_vertices` effettua il merge delle facce vicine, vertici e lati.

Con la funzione `@btime` valutiamo le prestazioni dal punto di vista temporale della funzione.

In [75]:
V, EV, FE = get_input()
@btime Lar.Arrangement.merge_vertices($Lar.Points(V),$Lar.ChainOp(EV),$Lar.ChainOp(FE),$1e-8)

([0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; … ; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087], sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 32), sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

# Ottimizazzione

In questa sezione rivisitiamo la funzione `merge_vertices` utilizzando la macro Julia `@async`. Tale macro avvolge un espressione in un `Task` e lo aggiunge alla coda dello scheduler della macchina locale.

In [76]:
function merge_vertices_revisited(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)

    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)

    # remove vertices congruent to a single representative
    todelete = []
    i = 1

    for vi in 1:vertsnum #non è possibile utilizzare @async
        if !(vi in todelete)
            nearvs =  Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs =  setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            i = i + 1
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]
    
    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)

    for ei in 1:edgenum #non è possibile utilizzare @async
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end

    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))
 
    etuple2idx = Dict{Tuple{Int, Int}, Int}()

    for ei in 1:nedgenum #non è possibile utilizzare @async
        begin
            nEV[ei, collect(nedges[ei])] .= 1
            nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    
    @async for e in 1:nedgenum 
        v1,v2 = findnz(nEV[e,:])[1]
        nEV[e,v1] = -1; nEV[e,v2] = 1
    end
 
    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]
 
 
    visited = []
    function filter_fn(face)
 
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)
 
        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end
 
    nfaces = filter(filter_fn, faces)
 
    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))
 
    @async for fi in 1:nfacenum
        @async for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end
 
    return Lar.Points(nV), nEV, nFE
 end

merge_vertices_revisited (generic function with 2 methods)

Utilizziamo la macro `@btime` per valutare le prestazioni dal punto di vista temporale e capire se le prestazioni della funzione `merge_vertices` sono migliorate.

In [77]:
V, EV, FE = get_input()
@btime merge_vertices_revisited(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

([0.6540618 0.2054992 0.2972308; 0.7142365 0.1455625 0.969203; … ; 0.2572286 -0.0086196 0.1785838; 0.4334306 0.1497748 0.5036087], sparse([1, 5, 9, 1, 6, 10, 2, 5, 11, 2  …  33, 27, 32, 34, 28, 31, 35, 28, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 32), sparse([1, 3, 1, 4, 2, 3, 2, 4, 1, 5  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, 1, -1, -1, -1, -1, 1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

In [78]:
V, EV, FE = get_input()
@benchmark merge_vertices_revisited(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-8)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):   79.958 μs …  38.350 ms  ┊ GC (min … max):  0.00% … 99.69%
 Time  (median):      83.750 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   101.994 μs ± 776.106 μs  ┊ GC (mean ± σ):  16.92% ±  2.23%

     ▁▄▇█▇▆▄▃▃▂▁                                                 
  ▁▂▅█████████████▇▆▅▄▄▄▃▃▃▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  80 μs            Histogram: frequency by time          103 μs <

 Memory estimate: 174.80 KiB, allocs estimate: 2622.

# Valutazione spatial_arrangment_1 con merge_vertices ottimizzato

In questa sezione aggiungiamo al codice del precedente metodo `spatial_arrangment_1` la funzione `merge_vertices_revisited` e rivisitiamo la funzione `spatial_arrangement_1_with_merge` utilizzando la macro `@async` la quale racchiude un Task e si occupa di far iniziare subito l’attività racchiusa nel task procedendo con qualsiasi operazione che segue la macro.

In [79]:
function spatial_arrangement_1_with_merge(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = Lar.Arrangement.frag_face(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = merge_vertices_revisited(rV, rEV, rFE)
  return rV, rEV, rFE
end

spatial_arrangement_1_with_merge (generic function with 2 methods)

Con la funzione `@btime` valutiamo le prestazioni dal punto di vista temporale della funzione.

In [80]:
V, EV, FE = get_input()
@btime spatial_arrangement_1_with_merge(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) 

([0.6540618000000001 0.2054992 0.2972308; 0.7142365000000002 0.1455625 0.969203; … ; 0.8049656 0.3844582 0.992188; 0.4810731000000001 0.3988786 0.9928307], sparse([1, 4, 14, 2, 5, 15, 1, 2, 7, 17  …  57, 61, 65, 53, 56, 63, 55, 58, 64, 66], [1, 1, 1, 2, 2, 2, 3, 3, 3, 3  …  36, 36, 36, 37, 37, 37, 38, 38, 38, 38], Int8[-1, -1, -1, -1, -1, -1, 1, 1, -1, -1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 67, 38), sparse([1, 4, 2, 5, 1, 6, 1, 7, 2, 8  …  25, 29, 27, 28, 26, 31, 32, 31, 30, 31], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  62, 62, 63, 63, 64, 64, 65, 66, 67, 67], Int8[1, 1, -1, -1, -1, 1, -1, 1, 1, 1  …  1, -1, -1, 1, 1, 1, 1, -1, -1, 1], 32, 67))

In [81]:
V, EV, FE = get_input()
@benchmark spatial_arrangement_1_with_merge(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),false) 

BenchmarkTools.Trial: 49 samples with 1 evaluation.
 Range (min … max):   59.096 ms … 214.124 ms  ┊ GC (min … max): 0.00% … 32.79%
 Time  (median):     100.706 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   103.487 ms ±  29.004 ms  ┊ GC (mean ± σ):  6.00% ± 10.40%

          ▅     ▂ █▅    ▂                                        
  █▁▁▅▅▅▅▅███▅▁██▅██▅████▁▅▅▁▁▅▁▁▁▁▁▁▅▅▁▁▁▅▁▁▁▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅ ▁
  59.1 ms          Histogram: frequency by time          214 ms <

 Memory estimate: 11.89 MiB, allocs estimate: 224615.

In questa sezione valutiamo la funzione `spatial_arrangment_1` con `merge_vertices` e `frag_face` ottimizzati

In [82]:
function spatial_arrangement_1_rev(
		V::Lar.Points,
		copEV::Lar.ChainOp,
		copFE::Lar.ChainOp, multiproc::Bool=false)

	# spaceindex computation
	FV = Lar.compute_FV( copEV, copFE )
	model = (convert(Lar.Points,V'), FV)
	sp_idx = Lar.spaceindex(model) # OK!!  tested symmetry of computed relation

	# initializations
	fs_num = size(copFE, 1)
  rV = Array{Float64,2}(undef,0,3)
  rEV = SparseArrays.spzeros(Int8,0,0)
  rFE = SparseArrays.spzeros(Int8,0,0)

	# multiprocessing of face fragmentation
    if (multiproc == true)
        in_chan = Distributed.RemoteChannel(()->Channel{Int64}(0))
        out_chan = Distributed.RemoteChannel(()->Channel{Tuple}(0))
        @async begin
            for sigma in 1:fs_num
                put!(in_chan, sigma)
            end
            for p in Distributed.workers()
                put!(in_chan, -1)
            end
        end
        for p in Distributed.workers()
            @async Base.remote_do(
                frag_face_channel, p, in_chan, out_chan, V, EV, FE, sp_idx)
        end
        for sigma in 1:fs_num
            rV, rEV, rFE = Lar.skel_merge(rV, rEV, rFE, take!(out_chan)...)
        end
    else
	# sequential (iterative) processing of face fragmentation
        for sigma in 1:fs_num
            #println("\n",sigma, "/", fs_num)
            nV, nEV, nFE = frag_face_parallelized(V, copEV, copFE, sp_idx, sigma)
		      	nV = convert(Lar.Points, nV)
            rV,rEV, rFE= Lar.skel_merge( rV,rEV,rFE,  nV,nEV,nFE )
            #rV=a;  #rEV=b;  #rFE=c
        end
    end
	# merging of close vertices, edges and faces (3D congruence)
	rV, rEV, rFE = merge_vertices_revisited(rV, rEV, rFE)
  return rV, rEV, rFE
end

spatial_arrangement_1_rev (generic function with 2 methods)

Con la funzione `@btime` valutiamo le prestazioni dal punto di vista temporale della funzione.

In [83]:
V, EV, FE = get_input()
@btime spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE)) 

([0.6540618000000001 0.2054992 0.2972308; 0.7142365000000002 0.1455625 0.969203; … ; 0.8049656 0.3844582 0.992188; 0.4810731000000001 0.3988786 0.9928307], sparse([1, 3, 9, 1, 4, 10, 2, 3, 11, 2  …  33, 29, 32, 34, 30, 31, 35, 30, 32, 36], [1, 1, 1, 2, 2, 2, 3, 3, 3, 4  …  21, 22, 22, 22, 23, 23, 23, 24, 24, 24], Int8[-1, -1, -1, 1, -1, -1, -1, 1, -1, 1  …  1, 1, -1, 1, -1, 1, 1, 1, 1, 1], 36, 24), sparse([1, 3, 1, 4, 1, 5, 1, 6, 2, 3  …  14, 18, 15, 17, 15, 18, 16, 17, 16, 18], [1, 1, 2, 2, 3, 3, 4, 4, 5, 5  …  32, 32, 33, 33, 34, 34, 35, 35, 36, 36], Int8[1, 1, -1, 1, -1, 1, 1, 1, 1, -1  …  1, -1, -1, -1, 1, -1, -1, 1, 1, 1], 18, 36))

In [84]:
V, EV, FE = get_input()
@benchmark spatial_arrangement_1_rev(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE)) 

BenchmarkTools.Trial: 1201 samples with 1 evaluation.
 Range (min … max):  3.466 ms … 22.092 ms  ┊ GC (min … max):  0.00% … 82.04%
 Time  (median):     3.557 ms              ┊ GC (median):     0.00%
 Time  (mean ± σ):   4.160 ms ±  3.023 ms  ┊ GC (mean ± σ):  13.67% ± 14.88%

  █                                                           
  ██▆▁▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▁▄▃▅▅▃▁▅▄▆▅▅▃▅ ▇
  3.47 ms      Histogram: log(frequency) by time     20.7 ms <

 Memory estimate: 4.55 MiB, allocs estimate: 82451.